# FLIP(01):  Advanced Data Science
**(Module F: MLflow)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.
- If you found any issue/bug for this document, please submit an issue at [tulip lab/flip01](https://github.com/tulip-lab/flip01/issues)


Prepared by and for 
**Student Members** |
2006-2022 [TULIP Lab](http://www.tulip.org.au)

---


# Session 04 - Deploy the Model to Seldon Core or KServe (experimental)



After training and testing our model, we are now ready to deploy it to production. MLflow allows you to serve your model using MLServer, which is already used as the core Python inference server in Kubernetes-native frameworks including Seldon Core and KServe (formerly known as KFServing). Therefore, we can leverage this support to build a Docker image compatible with these frameworks.

## Note
Note that this an optional step, which is currently only available for Python models. Besides this, it’s also worth noting that:

* This feature is experimental and is subject to change.

* MLServer requires Python 3.7 or above.

* This step requires some basic Kubernetes knowledge, including familiarity with kubectl.

To build a Docker image containing our model, we can use the mlflow models build-docker subcommand, alongside the *--enable-mlserver* flag. For example, to build a image named *my-docker-image*, we could do:

In [ ]:
mlflow models build-docker \
  -m /Users/mlflow/mlflow-prototype/mlruns/0/7c1a0d5c42844dcdb8f5191146925174/artifacts/model \
  -n my-docker-image \
  --enable-mlserver

Once we have our image built, the next step will be to deploy it to our cluster. One way to do this is by applying the respective Kubernetes manifests through the kubectl CLI:



In [ ]:
kubectl apply -f my-manifest.yaml


This step assumes that you’ve got kubectl access to a Kubernetes cluster already setup with Seldon Core. To read more on how to set this up, you can refer to the Seldon Core quickstart guide.

In [ ]:
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: mlflow-model
spec:
  predictors:
    - name: default
      annotations:
        seldon.io/no-engine: "true"
      graph:
        name: mlflow-model
        type: MODEL
      componentSpecs:
        - spec:
            containers:
              - name: mlflow-model
                image: my-docker-image
                imagePullPolicy: IfNotPresent
                securityContext:
                  runAsUser: 0
                ports:
                  - containerPort: 8080
                    name: http
                    protocol: TCP
                  - containerPort: 8081
                    name: grpc
                    protocol: TCP
